Import libraries

In [13]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score

import hopsworks

Get model from model registry

In [14]:
project = hopsworks.login()

# Get the model registry
mr = project.get_model_registry()

2025-01-06 01:30:21,590 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-06 01:30:21,601 INFO: Initializing external client
2025-01-06 01:30:21,602 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 01:30:23,683 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079


In [15]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="news_propagation_model",
    version=4,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [16]:
# Initialize the model
model = xgb.XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

XGBClassifier(base_score='5.141206E-1', booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['float', 'float', 'int', 'int', 'float', 'float',
                             'float', 'int', 'int', 'float', 'float'],
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Get features to predict

In [17]:
fs = project.get_feature_store()

In [18]:
news_propagation_fg = fs.get_feature_group(
    name="news_propagation",
    version=1,
)

In [19]:
selected_features = news_propagation_fg.select_except(["news_id"])

In [20]:
feature_view = fs.get_or_create_feature_view(
        name = 'news_propagation_feature_views',
        version = 1,
        labels=["label"],
        query=selected_features,
)

In [21]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get the untransformed and untransformed batch data for logging
batch_data = feature_view.get_batch_data()

# Drop the "datetime" column from the batch_data DataFrame along the specified axis (axis=1 means columns)
# batch_data = transformed_batch_data.drop(["news_id"], axis=1)

# Display the first 3 rows
batch_data.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 


,average_followers,average_follows,repost_total,post_total,repost_percentage,average_repost,average_favorite,news_lifetime,nb_users_10_hours,average_time_difference,retweet_percentage_1_hour
0,625.000000,438.000000,0,1,0.000000,0.000000,0.000000,0,1,0.0,1.000000
1,1410.833333,1103.416667,4,10,0.285714,0.285714,6.166667,63372,1,0.0,0.071429
2,536.454545,342.909091,979,1,0.998980,9.989796,499.000000,0,1,0.0,0.001020


In [22]:
len(batch_data)

254

In [28]:
# Use the retrieved XGBoost model to make predictions on the batch data
predictions = model.predict(batch_data.tail(1))

# Display the first five predictions
predictions

array([0])